In [45]:
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from rouge import Rouge

In [28]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [29]:
df = df.iloc[:300]

In [30]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


# Q1

In [5]:
model_name = 'multi-qa-mpnet-base-dot-v1'

In [6]:
embedding_model = SentenceTransformer(model_name)

In [7]:
answer_llm = df.iloc[0].answer_llm

In [9]:
v = embedding_model.encode(answer_llm)

In [12]:
v[0]

-0.42244655

# Q2

In [11]:
evaluations = []

In [14]:
def compute_similarity(record, model):
    v_llm = model.encode(record['answer_llm'])
    v_orig = model.encode(record['answer_orig'])
    return v_llm.dot(v_orig)

In [17]:
def compute_similarity_all_df(df, model, evaluations):
    for i in tqdm(range(len(df)), desc="Computing Similarities"):
        evaluations.append(compute_similarity(df.iloc[i], model))
    return evaluations

In [18]:
evaluations = compute_similarity_all_df(df, embedding_model, evaluations)

Computing Similarities: 100%|█| 300/300 [02:25<00:00,  


In [19]:
df1 = df.copy()
df1['cosine_sim'] = evaluations

In [20]:
df1['cosine_sim'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine_sim, dtype: float64

# Q3

In [31]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [32]:
def compute_similarity(record, model):
    v_llm = normalize_vector(model.encode(record['answer_llm']))
    v_orig = normalize_vector(model.encode(record['answer_orig']))
    return v_llm.dot(v_orig)

In [38]:
evaluations1 = []

In [39]:
evaluations1 = compute_similarity_all_df(df, embedding_model, evaluations1)

Computing Similarities: 100%|█| 300/300 [02:25<00:00,  


In [40]:
len(evaluations1)

300

In [42]:
df2 = df.copy()
df2['cosine_sim'] = evaluations1

In [43]:
df2['cosine_sim'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine_sim, dtype: float64

# Q4

In [48]:
df.head(11)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
5,"The course videos are pre-recorded, and you ca...","The course videos are pre-recorded, you can st...",39fda9f0,Are the course videos live or pre-recorded?,machine-learning-zoomcamp
6,You can start watching the course videos right...,"The course videos are pre-recorded, you can st...",39fda9f0,When can I start watching the course videos?,machine-learning-zoomcamp
7,"Yes, the live office hours sessions are recorded.","The course videos are pre-recorded, you can st...",39fda9f0,Are the live office hours sessions recorded?,machine-learning-zoomcamp
8,You can find the office hours sessions in the ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I find the office hours sessions?,machine-learning-zoomcamp
9,You can access the pre-recorded course videos ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I access the pre-recorded course vid...,machine-learning-zoomcamp


In [49]:
r = df.iloc[10]

In [50]:
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [51]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

# Q5

In [57]:
average_scores = {}
average_r = 0
average_p = 0
average_f = 0
for x in scores.items():
    # print(x[1])
    average_r += x[1]['r']
    average_p += x[1]['p']
    average_f += x[1]['f']
average_r /= 3
average_p /= 3
average_f /= 3
average_scores['r'] = average_r
average_scores['p'] = average_p
average_scores['f'] = average_f

In [58]:
average_scores

{'r': 0.35490035490035493, 'p': 0.35490035490035493, 'f': 0.35490034990035496}

# Q6

In [66]:
rouge_score_list = []
rouge_l_score_list = []
def compute_avg_rouge_score(df, rouge_scorer, score_list, l_score_list):
    for i in tqdm(range(len(df)), desc="Computing Rouge score"):
        r = df.iloc[i]
        scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
        rouge_1 = scores['rouge-1']['f']
        rouge_2 = scores['rouge-2']['f']
        rouge_l = scores['rouge-l']['f']
        l_score_list.append(rouge_l)
        rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
        score_list.append(rouge_avg)
    return score_list,l_score_list

In [67]:
rouge_score_list,rouge_l_score_list = compute_avg_rouge_score(df, rouge_scorer, rouge_score_list,rouge_l_score_list)

Computing Rouge score: 100%|█| 300/300 [00:00<00:00, 34


In [68]:
df3 = df.copy()
df3['rouge_score'] = rouge_score_list
df3['rouge_l_score'] = rouge_l_score_list
df3['rouge_score'].describe()

count    300.000000
mean       0.313205
std        0.158133
min        0.000000
25%        0.197358
50%        0.298640
75%        0.404169
max        0.813043
Name: rouge_score, dtype: float64

In [69]:
df3['rouge_l_score'].describe()

count    300.000000
mean       0.353807
std        0.162965
min        0.000000
25%        0.228032
50%        0.337792
75%        0.451613
max        0.850000
Name: rouge_l_score, dtype: float64

In [70]:
df3


,answer_llm,answer_orig,document,question,course,rouge_score,rouge_l_score
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.072882,0.095238
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.091435,0.093750
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.327658,0.389610
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0.150821,0.189189
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.098731,0.120219
...,...,...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp,0.604570,0.618182
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp,0.535991,0.557377
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp,0.618851,0.637168
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp,0.247252,0.304762
